### Integration script


This Jupyter notebook is used to visualize the program flow and the measured values. It combines the advantages of a user interface through easy handling and direct adaptation of the variables by the user, while enabling simple and direct visualization of the measurement results.
The individual code cells can be started independently of each other, but some of them build on each other (note the markers!).


In [ ]:
#load the required packages
%matplotlib ipympl

import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import thorlabs_apt as apt
import pyvisa as visa
import pandas as pd
import numpy as np
import time
import os
from scipy.optimize import curve_fit

To be able to control the correct rotation stage, the available devices are queried and printed out.

In [ ]:
devices = apt.list_available_devices()
print(devices)

The engine is defined by its serial number and various settings can be made.

In [ ]:
motor = apt.Motor(55380084)
#STAGE_UNITS_DEG = 2 #Stage units in degrees

In [ ]:
motor.identify()
home1 = motor.get_move_home_parameters()
print(home1)
motor.set_move_home_parameters(2,1,10,4) #werte von https://github.com/qpit/thorlabs_apt/issues/24
home2 = motor.get_move_home_parameters()
print(home2)
motor.move_home()

Selecting the right power meter based on the serial number of the console

In [ ]:
rm = visa.ResourceManager()
print(rm.list_resources())

The power meter is set up as follows. Various settings can be made.

In [ ]:
inst = rm.open_resource('USB0::0x1313::0x8078::P0029761::INSTR', timeout=1)
power_meter = ThorlabsPM100(inst=inst)
power_meter.configure.scalar.power() #quelle: https://pythonhosted.org/ThorlabsPM100/thorlabsPM100.html#main-commands 
print("Measurement type :", power_meter.getconfigure)

In [ ]:
#Initialization and settings of the fit for the measurements 
def sin2_func(x, A, B, C, D):
    return A * np.sin(B * x + C)**2 + D

def sin2_func_in_degrees(x, A, B, C, D):
    return A * np.sin(np.deg2rad(B * x + C))**2 + D

def get_min_values(measure_table, plot_filename, initial_guess=None):
    # Schätzungen für die Anfangswerte
    # initial_guess = [1e-4, 0.1, 0, 1e-5] #rad
    if initial_guess is None:
        initial_guess =  np.array([np.max(np.abs(measure_table['measure'])) - np.mean(measure_table['measure']), 1, 0, np.mean(measure_table['measure'])]) #grad

    sigma = measure_table['measure'] * 0.03
    plt.plot(measure_table['position'], sigma, label='Gewichte', color='red')
    plt.savefig(f"output/gewichte_{plot_filename.split('_')[1]}")
    plt.close()
    popt, pcov = curve_fit(sin2_func_in_degrees, measure_table['position'], measure_table['measure'], p0=initial_guess, sigma=sigma, maxfev = 10000)
    
    # Daten für den Fit berechnen
    fit_x = np.linspace(measure_table['position'].min(), measure_table['position'].max(), 1000)
    fit_y = sin2_func_in_degrees(fit_x, *popt)

    # Minimum des Fits finden
    min_index = np.argmin(fit_y)
    min_fit_pos = fit_x[min_index]
    min_fit_value = fit_y[min_index]

    # Gegebener minimaler Wert und Position aus Messung
    min_W_value = measure_table['measure'].min()
    min_pos_row = measure_table[measure_table['measure'] == min_W_value]
    min_pos = min_pos_row['position'].iloc[0]

    # Plot der Originaldaten und des Fits
    plt.figure(figsize=(10, 6))
    plt.scatter(measure_table['position'], measure_table['measure'], label='Originaldaten', color='blue')
    plt.plot(fit_x, fit_y, label='Gewichteter Sinus^2 Fit', color='red')
    plt.axvline(x=min_fit_pos, color='green', linestyle='--', label='Minimum Position')
    #plt.scatter(min_pos, min_W_value, color='orange', zorder=5, label='Measured Minimum Value')
    plt.axvline(x=min_pos, color='orange', linestyle='--', label='Measured Minimum Value')
    plt.title(f'Fit Min: {min_fit_value:.2e} \nbei {min_fit_pos:.2f}°; Messung Min: {min_W_value:.2e} \nbei {min_pos:.2f}°')
    plt.xlabel('Position')
    plt.ylabel('Measure')
    plt.legend()
    plt.title('Sinus^2 Fit zu den Messdaten')
    plt.savefig(plot_filename)
    plt.show()
    plt.close()

    return(initial_guess, min_fit_pos, min_fit_value, min_pos, min_W_value)

    print("Optimierte Parameter:", popt)
    print("Position des Minimums (Fit):", min_fit_pos)
    print("Wert des Minimums (Fit):", min_fit_value)
    print("Position des minimalen Wertes (Messung):", min_pos)
    print("Minimaler Wert (Messung):", min_W_value)

In [ ]:
def do_measurement_with_live_plot(start, stop, step_size, motor, power_meter): #Macht eine Messung 
    df = pd.DataFrame(columns=["position", "measure"])
    motor.move_to(start, blocking=True) #aufpassen, dass er nicht immer zu 0 fährt
    angle = start #pos

    while angle <= stop:
        measurements = [power_meter.read for _ in range(10)]
        mean_measurement = np.mean(measurements)
        df.loc[len(df)] = {"position": motor.position, "measure": mean_measurement}
        motor.move_by(step_size, blocking=True)
        angle += step_size #pos + step_size 
    return(df)

First, the 'start_measurement' function is defined.
This only needs to be successfully executed once, as long as the kernel does not need to be restarted.

In [ ]:
def start_measurement(start_input, stop_input, step_input, motor, power_meter, outputpath): #start_measurement führt do_measurement 3 mal aus
    
    # Überprüfen, ob die Eingabefelder nicht leer sind
    if start_input == '' or stop_input == '' or step_input == '':
        # abbrechen wenn leer
        print("Bitte geben Sie Werte für Startwinkel, Stopwinkel und Schrittgröße ein.")
        return

    try:
        start_angle = float(start_input)
        stop_angle = float(stop_input)
        step_size = float(step_input)
        #motor = int(motor_input)
    except ValueError:
        # abbrechen wenn ungültig
        print("Ungültige Eingaben. Bitte geben Sie gültige Zahlen ein.")
        return

    if step_size == 0:
        print("Step size darf nicht 0 sein!")
        return
    
    timestamp = time.strftime('%Y%m%d%H%M')
    with open(f"{timestamp}_log.txt", "w") as log:
        # Starten der Messung
        measure_table = do_measurement_with_live_plot(start_angle, stop_angle, step_size, motor, power_meter) #änderung 4
        log.write(f"{measure_table.to_string()}\n")
        measure_table.to_csv(os.path.join(outputpath,f"{timestamp}_01.csv"))

        # Minimalwert der Messung und zugehörigen Winkel finden
        initial_guess, min_fit_pos, min_fit_value, min_pos, min_W_value = get_min_values(measure_table, plot_filename=os.path.join(outputpath,f"{timestamp}_p1.png"))
        log.write(f"1. Minimal current value in W: {min_W_value}\n")
        log.write(f"1. Position corresponding to minimal current value: {min_pos}\n")
        log.write(f"1. Minimal current fit value in W: {min_fit_value}\n")
        log.write(f"1. Fitted position corresponding to minimal current value: {min_fit_pos}\n")
        motor.move_to(min_pos, blocking=True)
        #motor.move_to(min_fit_pos, blocking=True)
    #TODO mit in die Fit funktion übernehmen
        max_W_value = measure_table['measure'].max()
        max_pos_row = measure_table[measure_table['measure'] == max_W_value]
        max_pos = max_pos_row['position'].iloc[0]
        log.write(f"1. Maximal current value in W: {max_W_value}\n")
        log.write(f"1. Position corresponding to maximal current value: {max_pos}\n")

        # Messung um den minimalen Winkel (±10° in 1°-Schritten) durchführen
        measure_table = do_measurement_with_live_plot(start=min_pos - 8, stop=min_pos + 8, step_size=0.5, motor=motor, power_meter=power_meter)  #änderung 5
        log.write(f"{measure_table.to_string()}\n")
        measure_table.to_csv(os.path.join(outputpath,f"{timestamp}_02.csv"))
        # Minimalwert der zweiten Messung und zugehörigen Winkel finden
        _, min_fit_pos, min_fit_value, min_pos, min_W_value = get_min_values(measure_table, plot_filename=os.path.join(outputpath,f"{timestamp}_p2.png"), initial_guess=initial_guess)
        log.write(f"2. Minimal current value in W: {min_W_value}\n")
        log.write(f"2. Position corresponding to minimal current value: {min_pos}\n")
        log.write(f"2. Minimal current fit value in W: {min_fit_value}\n")
        log.write(f"2. Fitted position corresponding to minimal current value: {min_fit_pos}\n")
        motor.move_to(min_pos, blocking=True)

        # Messung um den minimalen Winkel (±5° in 0.5°-Schritten) durchführen
        measure_table = do_measurement_with_live_plot(start=min_pos - 2, stop=min_pos + 2, step_size=0.2, motor=motor, power_meter=power_meter) #änderung 6
        log.write(f"Measurements around the angle:\n")
        log.write(f"{measure_table.to_string()}\n")
        measure_table.to_csv(os.path.join(outputpath,f"{timestamp}_03.csv"))

        # Minimalwert der dritten Messung und zugehörigen Winkel finden
        _, min_fit_pos, min_fit_value, min_pos, min_W_value = get_min_values(measure_table, plot_filename=os.path.join(outputpath,f"{timestamp}_p3.png"), initial_guess=initial_guess)
        log.write(f"3. Minimal current value in W: {min_W_value}\n")
        log.write(f"3. Position corresponding to minimal current value: {min_pos}\n")
        log.write(f"3. Minimal current fit value in W: {min_fit_value}\n")
        log.write(f"3. Fitted position corresponding to minimal current value: {min_fit_pos}\n")
        motor.move_to(min_pos, blocking=True)

        # Motor zum neuen Winkel mit minimalem A-Wert bewegen
        motor.move_to(min_pos, blocking=True)
    print("Done")
    return(min_pos, min_fit_pos, max_pos)


In [ ]:
def start_justage(start, stop, step_size, motor, power_meter, outputpath):
    
    timestamp = time.strftime('%Y%m%d%H%M')
    with open(f"{timestamp}_log.txt", "w") as log:
        # Messung um den minimalen Winkel (±5° in 0.5°-Schritten) durchführen
        measurement_around_min_pos = do_measurement_with_live_plot(start=start, stop=stop, step_size=step_size, motor=motor, power_meter=power_meter) #änderung 6
        log.write(f"Measurements around the angle:\n")
        log.write(f"{measurement_around_min_pos.to_string()}\n")

        # Minimalwert der Messung und zugehörigen Winkel finden
        min_W_value = measurement_around_min_pos['measure'].min()
        min_pos_row = measurement_around_min_pos[measurement_around_min_pos['measure'] == min_W_value]
        min_pos = min_pos_row['position'].iloc[0]
        log.write(f"Just. Minimal current value in W: {min_W_value}\n")
        log.write(f"Just. Position corresponding to minimal current value: {min_pos}\n")

        max_W_value = measurement_around_min_pos['measure'].max()
        max_pos_row = measurement_around_min_pos[measurement_around_min_pos['measure'] == max_W_value]
        max_pos = max_pos_row['position'].iloc[0]
        log.write(f"1. Maximal current value in W: {max_W_value}\n")
        log.write(f"1. Position corresponding to maximal current value: {max_pos}\n")

        # Motor zum neuen Winkel mit minimalem A-Wert bewegen
        motor.move_to(min_pos, blocking=True)    
    return(min_pos)


---

### Outputpolarizer Alignment
\
The start parameters for the measurement can be set in the following code cell. These values are set for the first measurement and can be adjusted as needed.
In the definition of the start_measurement function in the code cell above

In [ ]:
# Eingabeparameter für das Outputpolarisator Alignment
start_input = '0'
stop_input = '360'
step_input = '1'
outputpath = os.path.join('C:\\','Users','clup','Desktop','Quellen','Messungen')
# Starten der Messung für das Outputpolarisator Alignment
min_pos,min_fit_pos,max_pos=start_measurement(start_input, stop_input, step_input, motor, power_meter, outputpath)
print(motor.position)
print(min_pos)

#wenn fit-Daten nicht nötig --> min_pos,_=start_measurement, dann ist 2 Variable leer

### Adjustment around the last measured minimum position

In [ ]:
# Eingabeparameter für das Outputpolarisator Alignment
start = min_pos-4
stop = min_pos+4
step_size = 0.5
outputpath = os.path.join('C:\\','Users','clup','Desktop','Quellen','output')
# Starten der Messung für das Outputpolarisator Alignment
min_pos=start_justage(start, stop, step_size, motor, power_meter, outputpath) #min_angel wird hier überschrieben
print(motor.position)
print(min_pos)
    

### Adjustment around the currently set position
If the motor is not homed, the angle values from here on will no longer match the display.

In [ ]:
# Eingabeparameter für das Outputpolarisator Alignment

pos = motor.position
start = pos
stop = pos+90
step_size = 1
outputpath = os.path.join('C:\\','Users','clup','Desktop','Quellen','output')
# Starten der Messung für das Outputpolarisator Alignment
min_pos=start_justage(start, stop, step_size, motor, power_meter, outputpath) 
print(motor.position)
print(min_pos)

###  Apply adhesive and lower the output polarizer onto the magnet


To determine whether the output polarizer was lowered onto the magnet without a wedge/angle or without accidentally twisting the polarizer, the operator has the option here to check the adjustment again for maximum.

---

### Waveplate Alignment
The start parameters for the measurement can be set in the following code cell. These values are set for the first measurement and can be adjusted as needed.
In the definition of the start_measurement function in the code cell above

In [ ]:
# Eingabeparameter für das Outputpolarisator Alignment
start_input = '0'
stop_input = '8'
step_input = '1'

outputpath = os.path.join("C:\\","Users","clup","Desktop","Quellen","waveplate")

# Starten der Messung für das Outputpolarisator Alignment
start_measurement(start_input, stop_input, step_input, motor, power_meter, outputpath)

## Apply adhesive and lower the waveplate onto the magnet
 

In [ ]:
# Eingabeparameter für das Outputpolarisator Alignment
start_input = 0
stop_input = 360
step_input = 1
outputpath = os.path.join('C:\\','Users','clup','Desktop','Quellen','Messungen')
# Starten der Messung für das Outputpolarisator Alignment
measure_table = do_measurement_with_live_plot(start_input, stop_input, step_input, motor, power_meter)
min_values, max_values = inextremo(measure_table)
print(motor.position)
print(min_values, max_values)

---